In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime

import requests

In [2]:
url_inflation_db = "https://github.com/matublaq/DataBases/raw/main/Inflation.db"
local_path = "Inflation.db"

response = requests.get(url_inflation_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Inflation database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Inflation database. Download failed. Status code {response.status_code}")

################################################################################################################

url_stockmarket_db = "https://github.com/matublaq/DataBases/raw/main/StockMarket.db"
local_path = "StockMarket.db"

response = requests.get(url_stockmarket_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Stock market database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Stock market database. Download failed. Status code {response.status_code}")

Inflation database. Downloaded successfully. Status code 200
Stock market database. Downloaded successfully. Status code 200


In [3]:
conn1 = sqlite3.connect('Inflation.db')
conn2 = sqlite3.connect('StockMarket.db')
cursor1 = conn1.cursor()
cursor2 = conn2.cursor()


cursor1.execute("SELECT name FROM Countries")
countries = cursor1.fetchall()
countries = [country[0] for country in countries]

cursor2.execute("SELECT name FROM Companies")
companies = cursor2.fetchall()
companies = [company[0] for company in companies]


conn1.close()
conn2.close()

print(countries, "\n", companies)

['Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Australia', 'Austria', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bolivia', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Côte d'Ivoire", 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Korea, Rep.', 'Kuwait', 'Lao PDR', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'L

<p style='font-size: 35px; text-align: center; color: violet;'>Functions</p>

In [4]:
def country_inflation(code):
    conn = sqlite3.connect('Inflation.db')
    cursor = conn.cursor()

    cursor.execute("SELECT id FROM Countries WHERE code = ?", (code,))
    id_usa = cursor.fetchall()[0]
    cursor.execute("SELECT year, inflation_rate FROM Inflation WHERE country_id = ?", id_usa)
    inflation = cursor.fetchall()

    conn.close()

    return pd.DataFrame(inflation, columns=['year', 'inflation_rate'])

def stock_quote_year_price(ticker):
    with sqlite3.connect('StockMarket.db') as conn:
        cursor = conn.cursor()

        cursor.execute("SELECT id FROM Companies WHERE ticker = ?", (ticker,))
        id_company = cursor.fetchall()[0]

        #Average price in dicember
        cursor.execute('''
                    SELECT strftime('%Y', date) as year, MAX(date) as max_date, close as price, volume
                    FROM Stock_quotes
                    WHERE company_id = ?
                    GROUP BY year
        ''', id_company)
        company_stock_year = cursor.fetchall()
        company_stock_year = pd.DataFrame(company_stock_year, columns=['year', 'max_date', 'price', 'volume'])
        company_stock_year.drop(columns=['max_date'], inplace=True)
        return company_stock_year

def price_inflation_adjusted(ticker, country_code):
    prices = stock_quote_year_price(ticker)
    inflations = country_inflation(country_code)
    prices['GROWTH'] = 0
    prices['real_price_adj'] = 0
    prices['inflation'] = 0
    prices['last_price_adj_inflation'] = 0

    today = datetime.datetime.now()
    for year in prices['year']:
        if year == str(today.year):
            prices.loc[prices['year'] == year, 'real_price_adj'] = prices.loc[prices['year'] == year, 'price']
            prices.loc[prices['year'] == year, 'inflation'] = prices.loc[prices['year'] == year, 'price']
            prices.loc[prices['year'] == year, 'last_price_adj_inflation'] = prices.loc[prices['year'] == year, 'price']
            return prices[['year', 'inflation', 'price', 'last_price_adj_inflation', 'real_price_adj', 'GROWTH']]
        
        inflation = inflations[inflations['year'] == int(year)]['inflation_rate'].values[0]

        if str(int(year)-1) in prices['year'].values:
            last_price = prices[prices['year'] == str(int(year)-1)]['price'].values[0]
            last_price_adj_inflation = last_price*((inflation/100) + 1)
            actual_price = prices[prices['year'] == year]['price'].values[0]
            
            diff_adj_actual = ((actual_price - last_price_adj_inflation)/last_price_adj_inflation)*100 #Creció o decreció.
            prices.loc[prices['year'] == year, 'GROWTH'] = diff_adj_actual.round(2)
            real_actual_price = last_price*((diff_adj_actual/100) + 1)
                         
            prices.loc[prices['year'] == year, 'real_price_adj'] = real_actual_price.round(3)
            prices.loc[prices['year'] == year, 'inflation'] = inflation
            prices.loc[prices['year'] == year, 'last_price_adj_inflation'] = last_price_adj_inflation.round(3)
        else:
            prices.loc[prices['year'] == year, 'real_price_adj'] = 0
            prices.loc[prices['year'] == year, 'inflation'] = 0
            prices.loc[prices['year'] == year, 'last_price_adj_inflation'] = 0

    return prices[['year', 'inflation', 'price', 'last_price_adj_inflation', 'real_price_adj', 'GROWTH']]


In [5]:
price_inflation_adjusted('AAPL', 'USA')

C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-40.97' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prices.loc[prices['year'] == year, 'GROWTH'] = diff_adj_actual.round(2)
C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.09' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prices.loc[prices['year'] == year, 'real_price_adj'] = real_actual_price.round(3)
C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.33' has dtype incompatible with int64, please explicitly cast t

,year,inflation,price,last_price_adj_inflation,real_price_adj,GROWTH
0,1980,0.00,0.152,0.000,0.000,0.00
1,1981,10.33,0.099,0.168,0.090,-40.97
2,1982,6.13,0.133,0.105,0.125,26.58
3,1983,3.21,0.109,0.137,0.106,-20.59
4,1984,4.30,0.130,0.114,0.125,14.35
5,1985,3.55,0.098,0.135,0.095,-27.20
6,1986,1.90,0.181,0.100,0.178,81.25
7,1987,3.66,0.375,0.188,0.362,99.87
8,1988,4.08,0.359,0.390,0.345,-8.02
9,1989,4.83,0.315,0.376,0.300,-16.30


In [6]:
inflations = country_inflation('USA')
inflations

,year,inflation_rate
0,1970,5.90
1,1971,4.29
2,1972,3.27
3,1973,6.18
4,1974,11.05
5,1975,9.14
6,1976,5.74
7,1977,6.50
8,1978,7.63
9,1979,11.25


In [7]:
prices = stock_quote_year_price('AAPL')
prices.columns

Index(['year', 'price', 'volume'], dtype='object')

In [8]:
prices_inflation = price_inflation_adjusted('MSFT', 'USA')
prices_inflation

C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '116.48' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prices.loc[prices['year'] == year, 'GROWTH'] = diff_adj_actual.round(2)
C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.364' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prices.loc[prices['year'] == year, 'real_price_adj'] = real_actual_price.round(3)
C:\Users\matia\AppData\Local\Temp\ipykernel_7404\1559896110.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.66' has dtype incompatible with int64, please explicitly cast t

,year,inflation,price,last_price_adj_inflation,real_price_adj,GROWTH
0,1986,0.00,0.168,0.000,0.000,0.00
1,1987,3.66,0.377,0.174,0.364,116.48
2,1988,4.08,0.370,0.392,0.355,-5.70
3,1989,4.83,0.604,0.388,0.576,55.72
4,1990,5.40,1.045,0.637,0.991,64.15
5,1991,4.23,2.318,1.089,2.224,112.82
6,1992,3.03,2.668,2.388,2.590,11.71
7,1993,2.95,2.520,2.747,2.448,-8.25
8,1994,2.61,3.820,2.586,3.723,47.73
9,1995,2.81,5.484,3.927,5.334,39.64


<p style="font-size: 25px; text-align: center;">Consultas generales</p>

In [9]:
conn = sqlite3.connect('StockMarket.db')
cursor = conn.cursor()
########################################################


########################################################
conn.commit()
conn.close()